In [ ]:
using MakieCore
import MakieCore: convert_arguments 

# This part would be the extension package (calls like lines, scatter, etc.)
using CairoMakie

import NamedTrajectories as NT

- convert_arguments cannot plot multiple lines? seemingly not without Makie (see nb 03). Stick with PointBased (MakieCore) or Series (Makie)?

Examples to follow
- https://github.com/JuliaGaussianProcesses/AbstractGPsMakie.jl/blob/main/src/recipes/gpsample.jl
- https://github.com/MakieOrg/Makie.jl/issues/837#issuecomment-846052033

In [ ]:
function convert_arguments(
    P::MakieCore.PointBased, 
    traj::NT.NamedTrajectory{<:Any},
    component::Int,
)
    times = NT.get_times(traj)
    positions = [(times[j], traj.data[component, j]) for j in eachindex(times)]
    return convert_arguments(P, positions)
end

# define the default plot type (preserve `plot` calls)
MakieCore.plottype(::NT.NamedTrajectory, ::Int) = MakieCore.Lines


In [ ]:
traj = rand(NT.NamedTrajectory, 10);

In [ ]:
f = Figure()
ax = Axis(f[1, 1])
[lines!(ax, traj, i, color=range(0, 1, length=traj.T), colormap=:Reds) for i in traj.components[:u]]
f

Notes
- Unsure why theme is not being set by the recipe. Use attributes to set it manually.

Little notes
- Makie defaults to Makie.Axis as the preferred axis.
- Can you set relational features of the figure (e.g. colgaps) in the recipe?
- How to deal with multiple symbols? (See nb 03 for Series). Recipes are only allowed one axis. SpecApi is experimental.

In [ ]:
@recipe(PlotComponents, traj, components) do scene
    Attributes(
        color=theme(scene, :linecolor),
        colormap=theme(scene, :colormap),
        colorrange=Makie.automatic,
        linestyle=theme(scene, :linestyle),
        linewidth=theme(scene, :linewidth),
        cycle=[:color],
    )
end

# Default is to plot on axis
Makie.preferred_axis_type(P::PlotComponents) = Makie.Axis

# Adds the ability to recall plot labels for a legend
Makie.get_plots(P::PlotComponents) = P.plots

# Dealing with a default plot of an existing component
function Makie.plot!(
    P::PlotComponents{<:Tuple{<:NT.NamedTrajectory, Symbol}};
    kwargs...
)
    lift(P[:traj], P[:components]) do traj, name
        times = NT.get_times(traj)
        for i in traj.components[name]
            lines!(
                P, times, traj.data[i, :], 
                label="$(name)$(i)",
                linewidth = P[:linewidth], 
                linestyle = P[:linestyle],
                color = P[:color],
                colorrange = P[:colorrange],
                colormap = P[:colormap],
                cycle = P[:cycle]
            )
        end
    end
    return P
end

# Dealing with a transformation to a new component
function Makie.plot!(
    P::PlotComponents{<:Tuple{<:NT.NamedTrajectory, <:Tuple{Symbol, Symbol, <:Function}}};
    kwargs...
)
    lift(P[:traj], P[:components]) do traj, transformation_data
        input_name, output_name, transformation = transformation_data
        times = NT.get_times(traj)

        # Descriptive error message
        output_data = try
            stack(transformation.(eachcol(traj[input_name])))
        catch
            throw(ArgumentError("Transformation $(input_name) -> $(output_name) failed."))
        end

        for (j, output_row) in enumerate(eachrow(output_data))
            lines!(
                P, times, output_row, 
                label="$(output_name)$(j)",
                linewidth = P[:linewidth], 
                linestyle = P[:linestyle],
                colorrange = P[:colorrange],
                colormap = P[:colormap],
                cycle = P[:cycle]
            )
        end
    end
    return P
end

- I'm not convinced that with_theme behaves correctly (doens't seem to reset if called in a block, but works if a function is passed)

In [ ]:
function test_plot()
    f = Figure()

    ax = Axis(f[1, 1])
    plotcomponents!(ax, traj, (:x, :y, x -> x .^ 2))
    Legend(f[1, 2], ax)

    ax = Axis(f[2, 1])
    plotcomponents!(ax, traj, :x)
    Legend(f[2, 2], ax)

    plotcomponents(f[3, 1], traj, :u,
        linewidth=5, linestyle=:dash,
        colormap=:Blues, colorrange=(0, 1),
        color=:red
    )
    return f
end

In [ ]:
test_plot()

In [ ]:
with_theme(test_plot, theme_minimal()) 

In [ ]:
with_theme(test_plot, test_theme) 